In [4]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import calendar

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [5]:
events_training = pd.read_csv('training.csv')
events_test = pd.read_csv('test.csv')
training = pd.read_csv('labels_training_set.csv')
test =  pd.read_csv('trocafone_kaggle_test.csv')

In [6]:
events_training.head()

,Unnamed: 0,timestamp,event,person,url,sku,model,condition,storage,color,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,label
0,0,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.00,iPhone 5s,Muito Bom,32GB,Cinza espacial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,2018-05-18 00:23:33,viewed product,ad93850f,NaN,318.00,iPhone 5s,Muito Bom,64GB,Prateado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2,2018-05-18 00:16:10,viewed product,ad93850f,NaN,"5,907.00",iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,3,2018-05-18 00:14:55,viewed product,ad93850f,NaN,"6,023.00",iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,4,2018-05-18 00:11:26,ad campaign hit,ad93850f,/comprar/iphone/iphone-5s,nan,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [7]:
events_test.head()

,Unnamed: 0,timestamp,event,person,url,sku,model,condition,storage,color,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,0,2018-05-18 00:11:59,viewed product,4886f805,NaN,"9,288.00",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2018-05-18 00:30:30,viewed product,4886f805,NaN,"9,288.00",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2018-05-18 00:07:23,search engine hit,4886f805,NaN,nan,NaN,NaN,NaN,NaN,...,Google,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2018-05-18 00:11:56,checkout,4886f805,NaN,"9,288.00",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2018-05-18 00:11:35,viewed product,4886f805,NaN,"9,287.00",Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# PRIMERO VAMOS A TRABAJAR CON EL DATAFRAME DE ENTRENAMIENTO

# Hacemos un feature engineering que es un one encoding modificaco. Se dice modificado porque en cada casillero ponemos la cantidad de cada. Uno de los eventos que hizo cada usuario.

In [8]:
sum_enconding=events_training.groupby(['person','event']).agg({'event':'count'}).unstack().fillna(0)
sum_enconding.columns=['ad campaign hit','brand listing','checkout','conversion','generic listing'\
                       ,'lead','search engine hit','searched products','staticpage','viewed product','visited site']

In [9]:
sum_enconding=sum_enconding.reset_index() 
sum_enconding.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,0008ed71,0.00,0.00,3.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,2.00
1,000c79fe,1.00,0.00,1.00,0.00,1.00,0.00,1.00,9.00,0.00,3.00,1.00
2,001802e4,5.00,0.00,1.00,0.00,4.00,0.00,0.00,4.00,0.00,4.00,1.00
3,0019e639,29.00,165.00,15.00,2.00,28.00,0.00,13.00,11.00,0.00,189.00,19.00
4,001b0bf9,0.00,1.00,2.00,1.00,1.00,0.00,0.00,0.00,0.00,2.00,0.00


# Aca le vamos a asignar a cada usuario cual fue el evento que mas hicieron y cuantas veces lo hicieron, y cual fue el evento que menos realizaron y cuantas veces hicieron el mismo.

In [10]:
datos_uno = events_training.loc[:,['person','event']]

In [11]:
cantidad_eventos = datos_uno.groupby('person').count().reset_index()
cantidad_eventos.columns=['person','event_count']
cantidad_eventos.head()

,person,event_count
0,0008ed71,6
1,000c79fe,17
2,001802e4,19
3,0019e639,471
4,001b0bf9,7


In [12]:
datos_dos = events_training.loc[:,['person','event','timestamp']]
datos_dos = datos_dos.groupby(['person','event']).count().reset_index()
datos_dos.columns=['person','event','cant_per_event']
datos_dos.head()

,person,event,cant_per_event
0,0008ed71,checkout,3
1,0008ed71,generic listing,1
2,0008ed71,visited site,2
3,000c79fe,ad campaign hit,1
4,000c79fe,checkout,1


In [13]:
maximo_evento = datos_dos.groupby('person').apply(lambda x: x.nlargest(1,'cant_per_event')).reset_index(drop=True)
maximo_evento.columns=['person','most_frequent_event','count_most_frequent_event']
maximo_evento.head()

,person,most_frequent_event,count_most_frequent_event
0,0008ed71,checkout,3
1,000c79fe,searched products,9
2,001802e4,ad campaign hit,5
3,0019e639,viewed product,189
4,001b0bf9,checkout,2


In [14]:
minimo_evento = datos_dos.groupby('person').apply(lambda x: x.nsmallest(1,'cant_per_event')).reset_index(drop=True)
minimo_evento.columns=['person','less_frequent_event','count_less_frequent_event']
minimo_evento.head()

,person,less_frequent_event,count_less_frequent_event
0,0008ed71,generic listing,1
1,000c79fe,ad campaign hit,1
2,001802e4,checkout,1
3,0019e639,conversion,2
4,001b0bf9,brand listing,1


# Agregamos un feature mas que es el promedio de la cantidad de cada evento que realizo el usuario.

In [15]:
media_evento = datos_dos.loc[:,['person','cant_per_event']]\
                .groupby('person').agg(lambda x: x.mean()).reset_index()
media_evento.columns=['person','mean_per_event']
media_evento.head(5)

,person,mean_per_event
0,0008ed71,2.00
1,000c79fe,2.43
2,001802e4,3.17
3,0019e639,52.33
4,001b0bf9,1.40


# Ahora agregamos los features en un solo dataframe.

In [16]:
valores = training.merge(maximo_evento,on='person',how='inner').merge(minimo_evento,on='person',how='inner')\
        .merge(sum_enconding,on='person',how='inner').merge(media_evento,on='person',how='inner')

In [17]:
valores.head()

,person,label,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,mean_per_event
0,0566e9c1,0,viewed product,23,checkout,1,6.00,3.00,1.00,1.00,15.00,0.00,1.00,0.00,1.00,23.00,17.00,7.56
1,6ec7ee77,0,ad campaign hit,1,ad campaign hit,1,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00
2,abe7a2fb,0,viewed product,31,checkout,1,9.00,14.00,1.00,0.00,9.00,0.00,4.00,6.00,0.00,31.00,22.00,12.00
3,34728364,0,viewed product,24,search engine hit,1,0.00,5.00,0.00,0.00,3.00,0.00,1.00,0.00,0.00,24.00,4.00,7.40
4,87ed62de,0,viewed product,9,visited site,1,5.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,9.00,1.00,4.25


# AHORA VOY A REPETIR TODO ESTO PARA EL DATAFRAME DE TESTEO

# Agrego el feature de one hot encodig modificado que hicimos antes.

In [18]:
sum_enconding_test=events_test.groupby(['person','event']).agg({'event':'count'}).unstack().fillna(0)
sum_enconding_test.columns=['ad campaign hit','brand listing','checkout','conversion','generic listing','lead','search engine hit','searched products','staticpage','viewed product','visited site']
sum_enconding_test=sum_enconding_test.reset_index() 
sum_enconding_test.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,00091926,15.00,25.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,372.00,34.00
1,00091a7a,1.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,1.00
2,000ba417,1.00,24.00,6.00,1.00,14.00,0.00,1.00,0.00,0.00,153.00,6.00
3,000e4d9e,19.00,17.00,1.00,0.00,17.00,0.00,5.00,0.00,0.00,339.00,13.00
4,000e619d,6.00,11.00,1.00,0.00,8.00,0.00,3.00,6.00,0.00,28.00,5.00


# Ahora calculo el evento maximo de cada usuario y su cantidad,y el evento minimo y su cantidad para cada usuario

In [19]:
datos_dos_test = events_test.loc[:,['person','event','timestamp']]
datos_dos_test = datos_dos_test.groupby(['person','event']).count().reset_index()
datos_dos_test.columns=['person','event','cant_per_event']
datos_dos_test.head()

,person,event,cant_per_event
0,00091926,ad campaign hit,15
1,00091926,brand listing,25
2,00091926,checkout,2
3,00091926,viewed product,372
4,00091926,visited site,34


In [20]:
maximo_evento_test = datos_dos_test.groupby('person').apply(lambda x: x.nlargest(1,'cant_per_event'))\
                .reset_index(drop=True)
maximo_evento_test.columns=['person','most_frequent_event','count_most_frequent_event']
maximo_evento_test.head()

,person,most_frequent_event,count_most_frequent_event
0,00091926,viewed product,372
1,00091a7a,brand listing,5
2,000ba417,viewed product,153
3,000e4d9e,viewed product,339
4,000e619d,viewed product,28


In [21]:
minimo_evento_test = datos_dos_test.groupby('person').apply(lambda x: x.nsmallest(1,'cant_per_event'))\
        .reset_index(drop=True)
minimo_evento_test.columns=['person','less_frequent_event','count_less_frequent_event']
minimo_evento_test.head()

,person,less_frequent_event,count_less_frequent_event
0,00091926,checkout,2
1,00091a7a,ad campaign hit,1
2,000ba417,ad campaign hit,1
3,000e4d9e,checkout,1
4,000e619d,checkout,1


# Ahora calcula la media de la cantidad de eventos para cada usuario.

In [22]:
media_evento_test = datos_dos_test.loc[:,['person','cant_per_event']]\
                .groupby('person').agg(lambda x: x.mean()).reset_index()
media_evento_test.columns=['person','mean_per_event']


In [23]:
media_evento_test.head()

,person,mean_per_event
0,00091926,89.60
1,00091a7a,2.50
2,000ba417,25.75
3,000e4d9e,58.71
4,000e619d,8.50


# Junto todos los features en un solo dataframe para cada usuario

In [24]:
valores_test = test.merge(maximo_evento_test,on='person',how='inner').merge(minimo_evento_test,on='person',how='inner')\
                .merge(media_evento_test,on='person',how='inner').merge(sum_enconding_test,on='person',how='inner')

In [25]:
valores_test.head()

,person,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,mean_per_event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,4886f805,viewed product,4,checkout,1,1.50,0.00,0.00,1.00,0.00,1.00,0.00,1.00,1.00,0.00,4.00,1.00
1,0297fc1e,viewed product,404,lead,1,70.88,29.00,4.00,7.00,0.00,21.00,1.00,0.00,6.00,0.00,404.00,95.00
2,2d681dd8,viewed product,13,ad campaign hit,1,3.25,1.00,5.00,1.00,0.00,1.00,0.00,2.00,1.00,0.00,13.00,2.00
3,cccea85e,viewed product,739,checkout,1,92.89,15.00,7.00,1.00,0.00,20.00,0.00,26.00,1.00,5.00,739.00,22.00
4,4c8a8b93,viewed product,177,checkout,2,32.12,14.00,8.00,2.00,0.00,14.00,0.00,13.00,9.00,0.00,177.00,20.00


# Guardo los dataframes en dos archivos csv que seran usados en otros programas.

In [32]:
valores.to_csv('feature_training_paso_uno.csv',index=False)
valores_test.to_csv('feature_test_paso_uno.csv',index=False)